# Part 5 

In [1]:

def remove_punctuation(sentence):
    hindi_punctuation = ['।', '॥', ',', '?', '"', "'", ';', '-', '(', ')', '[', ']', '{', '}', '...','!','.']
    for punctuation in hindi_punctuation:
        sentence = sentence.replace(punctuation, '')
    return sentence

In [2]:
with open('cs689_assignment.txt', 'r', encoding='utf-8') as file:
    Inp = file.readlines()

# for i in Inp:
#     print(i)

corpus=[]
ground_truth = []

i=0
while (i<(len(Inp)-1)):
    x = Inp[i]
    x=x[2:-1]
    corpus.append(x)
    ground_truth.append(Inp[i+1])
    i+=3

token = []
for i in ground_truth:
    l = i.split(',')
    x = l[len(l)-1]
    x=x[:-1]
    l=l[:-1]
    l.append(x)
    token.append(l)
    
# print(len(corpus) , len(token))
# print(corpus)
# print(token)



In [3]:
def remove_underscore( tokens):
    token=[]
    for i in tokens:
        j = i.lstrip('▁') 
        k = j.lstrip('#')
        token.append(k)
    return token
    


def find_True_positive(my_token ,  model_token):
    res = 0
    for i in my_token:
        if i in model_token:
            res+=1
    return res



def find_false_negative(my_token , model_token):
    res = 0
    for i in model_token:
        if i not in my_token:
            res+=1
    return res




def find_false_positive(my_token , model_token):
    res = 0
    for i in my_token:
        if i not in model_token:
            res+=1
    return res
    


            
    
    

# Unigram Model

In [4]:
import sentencepiece as spm
spm.SentencePieceTrainer.train('--input=hi_100.txt --model_prefix=m_unigram --vocab_size=2000 --model_type=unigram') 
sp_unigram = spm.SentencePieceProcessor() 
sp_unigram.load('m_unigram.model') 

True

In [5]:
tp=0
fn=0
fp=0

for i in range(len(corpus)):
    Input = remove_punctuation(corpus[i])
    m_token = sp_unigram.encode_as_pieces(Input)
    model_token = remove_underscore(m_token) 
    my_token = token[i]
#     print("model : ",model_token)
#     print("my_token : ",my_token)
    tp += find_True_positive(my_token , model_token)
    fn += find_false_negative(my_token , model_token)
    fp += find_false_positive(my_token , model_token)
    
    
precision = (tp)/(tp + fp)
recall = (tp)/(tp + fn)
f_score =  (2*precision * recall ) / (precision + recall)

print("Precision : ",precision)
print("Recall : ",recall)
print("F1-Score : ",f_score)
    
    

    

Precision :  0.04067796610169491
Recall :  0.012461059190031152
F1-Score :  0.019077901430842606


# BPE Model 

In [7]:
import sentencepiece as spm
spm.SentencePieceTrainer.train('--input=hi_100.txt --model_prefix=m_bpe --vocab_size=2000 --model_type=bpe') 
sp_bpe = spm.SentencePieceProcessor() 
sp_bpe.load('m_bpe.model')

True

In [8]:
tp=0
fn=0
fp=0

for i in range(len(corpus)):
    Input = remove_punctuation(corpus[i])
    m_tokens = sp_bpe.encode_as_pieces(Input)
    model_token = remove_underscore(m_tokens) 
    my_token = token[i]
#     print("model : ",model_token)
#     print("my_token : ",my_token)
    tp += find_True_positive(my_token , model_token)
    fn += find_false_negative(my_token , model_token)
    fp += find_false_positive(my_token , model_token)
    
precision = (tp)/(tp + fp)
recall = (tp)/(tp + fn)

if((precision + recall)==0):
    f_score=0
else:
    f_score =  (2*precision * recall ) / (precision + recall)

print("Precision : ",precision)
print("Recall : ",recall)
print("F1-Score : ",f_score)

Precision :  0.03728813559322034
Recall :  0.011
F1-Score :  0.016988416988416986


# M-BERT Model 

In [9]:
from pytorch_pretrained_bert import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False , max_len=1000)


In [10]:
tp=0
fn=0
fp=0

for i in range(len(corpus)):
    Input = remove_punctuation(corpus[i])
    m_tokens = tokenizer.tokenize(Input)
    model_token = remove_underscore(m_tokens) 
    my_token = token[i]
#     print("model : ",model_token)
#     print("my_token : ",my_token)
    tp += find_True_positive(my_token , model_token)
    fn += find_false_negative(my_token , model_token)
    fp += find_false_positive(my_token , model_token)
    
precision = (tp)/(tp + fp)
recall = (tp)/(tp + fn)

if((precision + recall)==0):
    f_score=0
else:
    f_score =  (2*precision * recall ) / (precision + recall)

print("Precision : ",precision)
print("Recall : ",recall)
print("F1-Score : ",f_score)

Precision :  0.03728813559322034
Recall :  0.009482758620689655
F1-Score :  0.015120274914089349


# Indic-BERT Model 

In [11]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from torch.utils import data
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm


tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert', use_fast=False)


In [12]:

tp=0
fn=0
fp=0

for i in range(len(corpus)):
    Input = remove_punctuation(corpus[i])
    t = tokenizer(Input , return_tensors="pt")
    m_tokens = tokenizer.convert_ids_to_tokens(t['input_ids'][0])
    model_token = remove_underscore(m_tokens) 
    my_token = token[i]
#     print("model : ",model_token)
#     print("my_token : ",my_token)
    tp += find_True_positive(my_token , model_token)
    fn += find_false_negative(my_token , model_token)
    fp += find_false_positive(my_token , model_token)
    
precision = (tp)/(tp + fp)
recall = (tp)/(tp + fn)

if((precision + recall)==0):
    f_score=0
else:
    f_score =  (2*precision * recall ) / (precision + recall)

print("Precision : ",precision)
print("Recall : ",recall)
print("F1-Score : ",f_score)




Precision :  0.05084745762711865
Recall :  0.018656716417910446
F1-Score :  0.0272975432211101


# White-Space Tokenizer

In [13]:

tp=0
fn=0
fp=0

for i in range(len(corpus)):
    Input = remove_punctuation(corpus[i])
    m_tokens = Input.split()
    model_token = remove_underscore(m_tokens) 
    my_token = token[i]
#     print("model : ",model_token)
#     print("my_token : ",my_token)
    tp += find_True_positive(my_token , model_token)
    fn += find_false_negative(my_token , model_token)
    fp += find_false_positive(my_token , model_token)
    
precision = (tp)/(tp + fp)
recall = (tp)/(tp + fn)

if((precision + recall)==0):
    f_score=0
else:
    f_score =  (2*precision * recall ) / (precision + recall)

print("Precision : ",precision)
print("Recall : ",recall)
print("F1-Score : ",f_score)




Precision :  0.061016949152542375
Recall :  0.0313588850174216
F1-Score :  0.041426927502876874


# Part - 6

## Model Evaluation Results

### Unigram Model:
- **Precision** :  0.04067796610169491
- **Recall** :  0.012461059190031152
- **F1-Score** :  0.019077901430842606
- This model is not very precise and misses a lot of relevant information. It doesn't do well at capturing the right words.

### BPE Model:
- **Precision** :  0.03728813559322034
- **Recall** :  0.011
- **F1-Score** :  0.016988416988416986
- Similar to the Unigram Model, this one also lacks precision and struggles to find important. This model also not very precise ans also misses relevent information.

### M-BERT Model:
- **Precision** :  0.05084745762711865
- **Recall** :  0.01989389920424403
- **F1-Score** :  0.02859866539561487
- This model performs better than the Unigram and BPE models. It's more accurate at finding the right words but still has room for improvement.

### Indic-BERT Model:
- **Precision** :  0.05084745762711865
- **Recall** :  0.018656716417910446
- **F1-Score** :  0.0272975432211101
- Similar to M-BERT, this model is good at finding relevant information but has some limitations.

### White-Space Tokenizer:
- **Precision** :  0.061016949152542375
- **Recall** :  0.0313588850174216
- **F1-Score** :  0.041426927502876874
- This model performs the best among all.The White-Space Tokenizer gives the highest precision, recall, and F-score among all tokenizers.It's more accurate and effective at capturing relevant information compared to the others.

### Summary : 
- while M-BERT and Indic-BERT Models are better than Unigram and BPE Models, the White-Space Tokenizer stands out as the most effective in terms of precision, recall, and F1-score. It does a better job of capturing the right words and is the most reliable option for tokenization. 

 - The problem with Hindi language arises because the computer sometimes doesn't understand how to correctly count and process the characters. This makes it hard for the models to break down the text into meaningful parts, which affects their ability to understand Hindi properly. So, fixing this issue is crucial for making sure the models can accurately handle Hindi text.
 
 - In the dataset used for training the model, there are specific words in Hindi known as Karaka words. the model may tokenize these Karaka words separately, that makes difference between the model's output and the ground truth. 
 
 - In Hindi, there are words that are made by putting two or more words together,  which is called "sandhi" and "Samasha". For models, figuring out where one word starts and another ends in these situations can be hard. ans also how to tokenize these kind of word can be hard. This makes it tricky for them to understand and work with Hindi text.
 
 
